In [ ]:
pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import psycopg2

In [ ]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd

In [ ]:
import json

In [ ]:
def fetch_copyright_data():
    try:
        connection = psycopg2.connect(
            dbname="fossology",
            user="fossy",
            password="fossy",
            host="localhost",
            port="5432"
        )
        cursor =  connection.cursor()

        sql = """
        SELECT DISTINCT ON(copyright_pk, UT.uploadtree_pk)
            copyright_pk,
            UT.uploadtree_pk AS uploadtree_pk,
            (CASE WHEN (CE.content IS NULL OR CE.content = '') THEN C.content ELSE CE.content END) AS content,
            (CASE WHEN (CE.hash IS NULL OR CE.hash = '') THEN C.hash ELSE CE.hash END) AS hash,
            C.agent_fk AS agent_fk,
            C.is_enabled AS original_is_enabled,
            CE.is_enabled AS modified_is_enabled
        FROM copyright C
        INNER JOIN uploadtree UT on C.pfile_fk = UT.pfile_fk
        LEFT JOIN copyright_event CE ON CE.copyright_fk = C.copyright_pk
            AND CE.uploadtree_fk = UT.uploadtree_pk
        WHERE C.content IS NOT NULL
            AND C.content != ''
        ORDER BY copyright_pk, UT.uploadtree_pk, content DESC;
        """

        cursor.execute(sql)

        records = cursor.fetchall()
        #print(records)
        results = []
        for record in records:
            result = {
                "content": record[2],
                "is_enabled": record[5]
            }
            results.append(result)
        df = pd.DataFrame(results)



    except Exception as error:
        print(f"Error: {error}")
    finally:
        print(df)
        # Save the dataframe to a CSV file
        df.to_csv('copyrights.csv', index=False)
        if connection:
            cursor.close()
            connection.close()

In [ ]:
__name__ == "__main__"
fetch_copyright_data()

                                                 content  is_enabled
0      Copyright (c) Facebook, Inc. and its affiliate...        True
1      Copyright (c) Facebook, Inc. and its affiliate...        True
2      Copyright (c) Facebook, Inc. and its affiliate...        True
3      Copyright (c) Facebook, Inc. and its affiliate...        True
4      Copyright (c) Facebook, Inc. and its affiliate...        True
...                                                  ...         ...
22150  Copyright 2015 The TensorFlow Authors. All Rig...        True
22151  Copyright 2015 The TensorFlow Authors. All Rig...        True
22152  Copyright 2015 The TensorFlow Authors. All Rig...        True
22153  Copyright 2022 The TensorFlow Authors. All Rig...        True
22154  Copyright 2021 The TensorFlow Authors. All Rig...        True

[22155 rows x 2 columns]
